In [1]:
from satproc.postprocess.polygonize import polygonize 
from satproc.filter import filter_by_max_prob

In [3]:
!mkdir ./data_results/filtered/

In [6]:
filter_by_max_prob(
    input_dir='./data_results/3D/600_600/more_labels',
    output_dir='./data_results/3D/filtered/more_labels',
    threshold=0.5
    )

100%|##########| 5102/5102 [00:13<00:00, 369.89it/s]


In [7]:
input_path = './data_results/3D/filtered/more_labels'
output_path = './data_results/poligonized/piscinas_600x600_thr05_morelabels.gpkg'

polygonize(
    threshold=0.5,
    input_dir=input_path,
    output=output_path
    )

Dissolve: 100%|##########| 222/222 [01:01<00:00,  3.58it/s]
/home/guillermo/anaconda3/envs/unet/lib/python3.10/site-packages/satproc/postprocess/polygonize.py:132: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if shp.type == "MultiPolygon":


In [8]:
# filtrar por area minima
src_file = output_path
dst_file = output_path[:-5] + "_utm.gpkg"

!ogr2ogr -s_srs EPSG:32721 -a_srs EPSG:32721 -f 'GPKG' $dst_file $src_file

In [9]:
min_area = 5
input_ = dst_file
print(input_)
output_ = input_[:-5] + "_up5.gpkg"
print(output_)

./data_results/poligonized/piscinas_600x600_thr05_morelabels_utm.gpkg
./data_results/poligonized/piscinas_600x600_thr05_morelabels_utm_up5.gpkg


In [11]:
!ogr2ogr \
    -t_srs EPSG:32721 \
    -f "GPKG" \
    -sql "SELECT * FROM piscinas_600x600_thr05_morelabels WHERE ST_Area(geom) > $min_area" \
    -dialect SQLITE \
    -nln results \
    $output_ \
    $input_